In [2]:
# Create Spark Context
from pyspark import SparkContext
from pyspark.sql import SparkSession, functions
from pyspark.sql.functions import col, split

import requests
#sc = SparkContext()
spark = (SparkSession
         .builder
         .getOrCreate()
         )
sc = spark.sparkContext

def download(date, hour):
    prefix = date + '-' + str(hour)
    url = 'https://data.gharchive.org/' + prefix + '.json.gz'
    path = prefix + '.json.gz'
    #response = requests.get(url, stream=True)
    #open(path, 'wb').write(response.content)
    return path


file = download('2022-09-19', '12')
json = spark.read.json(file)
json.createOrReplaceTempView("github")



eventsDF = spark.sql("SELECT type,repo.name as owner_repo,actor.login as submitter FROM github") \
        .withColumn("owner", split(col("owner_repo"),"/").getItem(0)) \
        .withColumn("repo", split(col("owner_repo"),"/").getItem(1)) \
        .drop("owner_repo")

                           
# List of Developers that own more than one repository;
eventsDF.drop('submitter','type').distinct().groupBy('owner').count().where("count > 1").orderBy(col('count').desc())


+-----------------+--------------------+
|            owner|                repo|
+-----------------+--------------------+
|        getpremia|         premia-demo|
|       urbanailab|urbanailab.github.io|
|       afzalali15|      FlutterTopFive|
|        alamayreh|     VIPPGeo_Dataset|
|    virtualstores|            ios-wifi|
|       wiredtiger|          wiredtiger|
|         Oztechan|                 CCC|
|           pallih|                 rss|
|         Su-Kyung|            Su-Kyung|
|          lisi4ok|         phalcon-api|
|          elastic|        integrations|
|       ashish4907|              Spring|
|  spring-projects|     spring-data-jpa|
|        saucelabs|      saucerest-java|
|          cdklabs| construct-hub-probe|
| keshavjindal1235|           project20|
|denniszhao-github|              config|
|            zcash|             orchard|
|    fresh-coupons|  fresh-coupons-data|
|       ZeroDragon|          ZeroDragon|
+-----------------+--------------------+
only showing top